In [2]:
# !wget https://huggingface.co/datasets/mesolitica/mixtral-magicoder/resolve/main/data/python-00000-of-00001.jsonl
# !wget https://huggingface.co/datasets/mesolitica/mixtral-magicoder/resolve/main/data/pythonanalytics-00000-of-00001.jsonl

In [3]:
# !wget https://huggingface.co/datasets/m-a-p/Code-Feedback/resolve/main/Code-Feedback.jsonl
# !wget https://huggingface.co/datasets/teknium/OpenHermes-2.5/resolve/main/openhermes2_5.json

In [4]:
import json

filtered = []
with open('Code-Feedback.jsonl') as fopen:
    for l in fopen:
        l = json.loads(l)
        if '```python' in l['messages'][1]['content']:
            filtered.append(l)

In [5]:
len(filtered)

40808

In [6]:
!mkdir code-feedback

mkdir: cannot create directory ‘code-feedback’: File exists


In [7]:
from agent import ClientJupyterKernel, Agent, Generator
import os

def answer(q, i):
    filename = f'code-feedback/{i}.json'
    if os.path.exists(filename):
        return
    
    jupyter_kernel_url = 'http://localhost:8081/execute'
    openai_base_url = 'http://localhost:8005/v1'
    model_name = 'xingyaoww/CodeActAgent-Mistral-7b-v0.1'

    generator = Generator(model_name, openai_base_url)
    prompt = q['messages'][0]['content']
    
    r = None
    agent = None
    for k in range(2):
        try:
            CONV_ID = f'code-feedback-{i}-{k}'
            code_executor = ClientJupyterKernel(jupyter_kernel_url, CONV_ID)
            agent = Agent(generator, code_executor, conv_id=CONV_ID, silent = True)
            r = agent.run_prompt(prompt, n_max_executions = 10)
            agent.code_executor.shutdown()
            if '<execute>' in r[2]['content']:
                break
        except Exception as e:
            print(e)
            if agent is not None:
                agent.code_executor.shutdown()
            pass
    with open(filename, 'w') as fopen:
        json.dump({'original': q, 'result': r}, fopen)

In [8]:
def consumer(queue, name):
    while True:
        if queue.qsize() == 0:
            break
        item = queue.get()
        answer(*item)
    print(f'consumer {name} done')

In [9]:
urls = [(q, no) for no, q in enumerate(filtered)]

In [11]:
!docker rm $(docker ps -a -q --filter="name=conv-code-feedback*") -f

"docker rm" requires at least 1 argument.
See 'docker rm --help'.

Usage:  docker rm [OPTIONS] CONTAINER [CONTAINER...]

Remove one or more containers


In [12]:
# !rm code-feedback/210.json
answer(*urls[2])

In [13]:
# !rm code-feedback/*.json

In [14]:
# with open('code-feedback/78.json') as fopen:
#     for r in json.load(fopen)['result']:
#         print(r)

In [15]:
from threading import Thread
from queue import Queue
from tqdm import tqdm

queue = Queue()
for u in urls:
    queue.put(u)
    
ori_size = queue.qsize()

In [ ]:
max_worker = 50
consumers = [Thread(target=consumer, args=(queue,i)) for i in range(max_worker)]
for i in range(len(consumers)):
    consumers[i].start()
    
pbar = tqdm(total=ori_size)
last_size = 0
while True:
    size = queue.qsize()
    if size == 0:
        break
    left = ori_size - size
    minus = left - last_size
    if minus > 0:
        pbar.update(minus)
        last_size += minus

pbar.close()

  0%|                                                                                                                          | 0/40808 [00:00<?, ?it/s]

Expecting value: line 1 column 1 (char 0)Expecting value: line 1 column 1 (char 0)

Expecting value: line 1 column 1 (char 0)Expecting value: line 1 column 1 (char 0)

Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)Expecting value: line 1 column 1 (char 0)

Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
New kernel created for conversation code-feedback-1157-0
New kernel created for conversation code-feedback-1136-0
New kernel created for conversation code-feedback-1123-0
New kernel created for conversation code-feedback-1143-0
New kernel created for conversation code-feedback-1142-0
New kernel created for conversation code-feedback-1117-0
New kernel created for conversation code-feedback-1115-0
New kernel created for conversation code-feedback-1100-0
New kernel created for conversation code-feedback-1155-0
New kernel created for conversation code-feedback-1118-0
New kernel created for conversation code-feedback-11

  3%|███                                                                                                          | 1163/40808 [02:42<1:36:41,  6.83it/s]

New kernel created for conversation code-feedback-1140-1New kernel created for conversation code-feedback-1147-1
New kernel created for conversation code-feedback-1137-1



  3%|███                                                                                                          | 1165/40808 [02:45<1:39:43,  6.63it/s]

...

New kernel created for conversation code-feedback-1128-1
New kernel created for conversation code-feedback-1160-0


  3%|███▏                                                                                                         | 1186/40808 [03:52<8:57:14,  1.23it/s]

New kernel created for conversation code-feedback-1171-0
New kernel created for conversation code-feedback-1163-0New kernel created for conversation code-feedback-1173-0
New kernel created for conversation code-feedback-1170-0
New kernel created for conversation code-feedback-1166-0
New kernel created for conversation code-feedback-1165-0
New kernel created for conversation code-feedback-1172-0New kernel created for conversation code-feedback-1174-0




  3%|███▏                                                                                                         | 1193/40808 [03:58<9:05:50,  1.21it/s]

New kernel created for conversation code-feedback-1164-0New kernel created for conversation code-feedback-1168-0

New kernel created for conversation code-feedback-1161-1New kernel created for conversation code-feedback-1181-0

